In [ ]:
# %% Colab Setup and Environment Preparation
%%capture
!pip install ultralytics albumentations matplotlib seaborn opencv-python-headless scikit-learn --quiet
!sudo apt-get install tree -qq

In [ ]:
import os
import cv2
import time
import numpy as np
import albumentations as A
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from pathlib import Path
from tqdm.auto import tqdm
import random
import shutil

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

# Mount Google Drive for model saving
drive.mount('/content/drive')

In [ ]:
# %% Dataset Configuration - Optimized for T4 GPU
class CFG:
    # Directory structure
    BASE_DIR = "/content/triangle_dataset"
    IMAGE_DIR = os.path.join(BASE_DIR, "images")
    LABEL_DIR = os.path.join(BASE_DIR, "labels")

    # Dataset parameters
    TOTAL_IMAGES = 1500  # (800 train, 350 val, 350 test)
    IMG_SIZE = 640
    TEST_SIZE = 0.23
    VAL_SIZE = 0.23
    NEGATIVE_RATIO = 0.2  # 20% images without triangles

    # Image variety parameters
    BG_TYPES = ['solid', 'gradient', 'noise', 'texture']
    BG_PROBS = [0.25, 0.3, 0.25, 0.2]

    # Triangle variety parameters
    MIN_TRIANGLE_AREA = 900
    MAX_TRIANGLES = 5
    TRIANGLE_TYPES = ['regular', 'thin', 'obtuse', 'right']

    # YOLO training
    MODEL_TYPE = "yolov8n.pt"  # Using nano model for speed
    EPOCHS = 100
    BATCH = 16
    PATIENCE = 20
    WORKERS = 2

    # Augmentation strengths
    AUG_ROTATE_LIMIT = 45
    AUG_SCALE_LIMIT = 0.2

    SAVE_DIR = "/content/drive/MyDrive/triangle_detection_rerun"

    # Visualizations
    PLOT_METRICS = True
    CONFUSION_MATRIX = True
    TEST_SAMPLES = 8

# Create directories
for dir_path in [CFG.BASE_DIR, CFG.IMAGE_DIR, CFG.LABEL_DIR]:
    os.makedirs(dir_path, exist_ok=True)

In [ ]:
# %% Enhanced Synthetic Dataset Generation
def generate_background():
    """Create diverse backgrounds with 4 different types"""
    bg_type = np.random.choice(CFG.BG_TYPES, p=CFG.BG_PROBS)
    img = np.zeros((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), dtype=np.uint8)

    if bg_type == 'solid':
        # Solid color with slight noise for realism
        base_color = np.random.randint(0, 256, 3)
        noise = np.random.randint(-20, 20, (CFG.IMG_SIZE, CFG.IMG_SIZE, 3))
        img = np.clip(np.full((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), base_color) + noise, 0, 255).astype(np.uint8)

    elif bg_type == 'gradient':
        if np.random.random() < 0.7:
            angle = np.random.randint(0, 360)
            c1, c2 = np.random.randint(0, 256, 3), np.random.randint(0, 256, 3)

            y, x = np.mgrid[0:CFG.IMG_SIZE, 0:CFG.IMG_SIZE]
            if angle < 90:
                mask = (x + y) / (2 * CFG.IMG_SIZE)
            elif angle < 180:
                mask = (CFG.IMG_SIZE - x + y) / (2 * CFG.IMG_SIZE)
            elif angle < 270:
                mask = (CFG.IMG_SIZE - x + CFG.IMG_SIZE - y) / (2 * CFG.IMG_SIZE)
            else:
                mask = (x + CFG.IMG_SIZE - y) / (2 * CFG.IMG_SIZE)

            for i in range(3):
                img[:,:,i] = (c1[i] * (1 - mask) + c2[i] * mask).astype(np.uint8)

        else:
            c1, c2 = np.random.randint(0, 256, 3), np.random.randint(0, 256, 3)
            center = (CFG.IMG_SIZE // 2, CFG.IMG_SIZE // 2)
            y, x = np.mgrid[0:CFG.IMG_SIZE, 0:CFG.IMG_SIZE]

            dist = np.sqrt((x - center[0])**2 + (y - center[1])**2)
            max_dist = np.sqrt(2) * CFG.IMG_SIZE / 2
            mask = np.clip(dist / max_dist, 0, 1)

            for i in range(3):
                img[:,:,i] = (c1[i] * (1 - mask) + c2[i] * mask).astype(np.uint8)

    elif bg_type == 'noise':
        noise_type = np.random.choice(['uniform', 'gaussian', 'salt_pepper'])

        if noise_type == 'uniform':
            img = np.random.randint(0, 256, (CFG.IMG_SIZE, CFG.IMG_SIZE, 3)).astype(np.uint8)

        elif noise_type == 'gaussian':
            mean = np.random.randint(0, 256)
            std = np.random.randint(5, 60)
            img = np.random.normal(mean, std, (CFG.IMG_SIZE, CFG.IMG_SIZE, 3))
            img = np.clip(img, 0, 255).astype(np.uint8)

        else:
            img = np.random.randint(30, 200, (CFG.IMG_SIZE, CFG.IMG_SIZE, 3)).astype(np.uint8)
            mask = np.random.random((CFG.IMG_SIZE, CFG.IMG_SIZE)) < 0.05
            img[mask] = 255
            mask = np.random.random((CFG.IMG_SIZE, CFG.IMG_SIZE)) < 0.05
            img[mask] = 0

    else:
        texture_type = np.random.choice(['grid', 'checker', 'voronoi'])

        if texture_type == 'grid':
            grid_size = np.random.randint(20, 100)
            color1 = np.random.randint(0, 256, 3)
            color2 = np.random.randint(0, 256, 3)

            x_grid = (np.arange(CFG.IMG_SIZE) // grid_size) % 2
            y_grid = (np.arange(CFG.IMG_SIZE) // grid_size) % 2
            grid = np.logical_xor.outer(x_grid, y_grid).astype(int)

            for i in range(3):
                img[:,:,i] = grid * color1[i] + (1 - grid) * color2[i]

        elif texture_type == 'checker':
            checker_size = np.random.randint(30, 120)
            color1 = np.random.randint(0, 256, 3)
            color2 = np.random.randint(0, 256, 3)

            x_check = (np.arange(CFG.IMG_SIZE) // checker_size) % 2
            y_check = (np.arange(CFG.IMG_SIZE) // checker_size) % 2
            checker = np.logical_xor.outer(x_check, y_check).astype(int)

            for i in range(3):
                img[:,:,i] = checker * color1[i] + (1 - checker) * color2[i]

        else:
            num_centers = np.random.randint(5, 20)
            centers = np.random.randint(0, CFG.IMG_SIZE, (num_centers, 2))
            colors = np.random.randint(0, 256, (num_centers, 3))

            y, x = np.mgrid[0:CFG.IMG_SIZE, 0:CFG.IMG_SIZE]

            # Calculate distances to each center
            distances = np.zeros((num_centers, CFG.IMG_SIZE, CFG.IMG_SIZE))
            for i in range(num_centers):
                distances[i] = np.sqrt((x - centers[i, 0])**2 + (y - centers[i, 1])**2)

            # Find closest center
            closest = np.argmin(distances, axis=0)

            # Assign colors
            for i in range(num_centers):
                mask = (closest == i)
                img[mask] = colors[i]

    return img

def create_triangle(triangle_type='random'):
    """Generate random triangle coordinates based on type"""
    if triangle_type == 'random':
        triangle_type = np.random.choice(CFG.TRIANGLE_TYPES)

    min_area = CFG.MIN_TRIANGLE_AREA
    margin = 10
    max_attempts = 100

    for _ in range(max_attempts):
        size = np.random.randint(50, CFG.IMG_SIZE // 2)
        x = np.random.randint(margin, CFG.IMG_SIZE - size - margin)
        y = np.random.randint(margin, CFG.IMG_SIZE - size - margin)

        if triangle_type == 'regular':
            angles = np.sort(np.random.uniform(0, 2*np.pi, 3))
            pts = np.array([
                [x + int(size * np.cos(a)), y + int(size * np.sin(a))]
                for a in angles
            ])
        elif triangle_type == 'thin':
            pts = np.array([
                [x, y],
                [x + np.random.randint(size//4, size//2), y + size],
                [x + np.random.randint(size//2, size), y]
            ])
        elif triangle_type == 'obtuse':
            pts = np.array([
                [x, y],
                [x + size, y],
                [x + np.random.randint(size//4, 3*size//4), y - np.random.randint(size//2, size)]
            ])
        else:  # right
            pts = np.array([
                [x, y],
                [x + size, y],
                [x, y + size]
            ])

        # Apply random rotation
        if np.random.random() < 0.7:
            center = tuple(map(int, pts.mean(axis=0)))
            rotation_matrix = cv2.getRotationMatrix2D(center, np.random.randint(0, 360), 1.0)
            pts = np.hstack((pts, np.ones((3, 1), dtype=np.float32)))
            pts = (rotation_matrix @ pts.T).T
            pts = np.clip(pts, margin, CFG.IMG_SIZE - margin).astype(int)

        # Check if area is sufficient
        area = cv2.contourArea(pts)
        if area >= min_area:
            return pts

    # fallback simple triangle
    center_x = CFG.IMG_SIZE // 2
    center_y = CFG.IMG_SIZE // 2
    size = np.random.randint(40, CFG.IMG_SIZE // 3)

    return np.array([
        [center_x, center_y - size // 2],
        [center_x - size // 2, center_y + size // 2],
        [center_x + size // 2, center_y + size // 2]
    ])

def get_triangle_color(background):
    """Generate a triangle color that contrasts with the background"""
    bg_center = background[background.shape[0]//2, background.shape[1]//2]

    # Determine if background is light or dark
    is_dark = np.mean(bg_center) < 128

    if is_dark:
        # Light triangle on dark background
        base = np.random.randint(150, 256, 3)
    else:
        # Dark triangle on light background
        base = np.random.randint(0, 100, 3)

    # Add some randomness
    color = base + np.random.randint(-30, 30, 3)
    return np.clip(color, 0, 255).tolist()

def draw_triangle_with_effects(img, triangle, color):
    """Draw a triangle with optional effects for realism"""
    # Create a copy for blending
    img_copy = img.copy()

    # Fill the triangle
    cv2.fillPoly(img_copy, [triangle], color)

    # Apply effects
    effect = np.random.choice(['none', 'border', 'gradient', 'texture'], p=[0.4, 0.3, 0.2, 0.1])

    if effect == 'border':
        # Add border
        border_color = np.array(color) * 0.7
        cv2.polylines(img_copy, [triangle], True, border_color.astype(int).tolist(), thickness=np.random.randint(1, 5))

    elif effect == 'gradient':
        mask = np.zeros_like(img)
        cv2.fillPoly(mask, [triangle], (255, 255, 255))

        gradient = np.zeros_like(img)
        pt1 = tuple(triangle[0])
        pt2 = tuple(triangle[2])
        cv2.line(gradient, pt1, pt2, (50, 50, 50), 5)

        gradient_strength = np.random.uniform(0.1, 0.3)
        img_copy = cv2.addWeighted(img_copy, 1, cv2.bitwise_and(gradient, mask), gradient_strength, 0)

    elif effect == 'texture':
        # Add texture to triangle
        texture_mask = np.zeros((CFG.IMG_SIZE, CFG.IMG_SIZE), dtype=np.uint8)
        cv2.fillPoly(texture_mask, [triangle], 255)

        # Create noise texture
        noise = np.random.randint(0, 50, (CFG.IMG_SIZE, CFG.IMG_SIZE))

        # Apply noise where mask is active
        for c in range(3):
            channel = img_copy[:,:,c]
            channel[texture_mask == 255] = np.clip(channel[texture_mask == 255] + noise[texture_mask == 255] - 25, 0, 255)
            img_copy[:,:,c] = channel

    alpha = np.random.uniform(0.85, 1.0)
    img = cv2.addWeighted(img, 1-alpha, img_copy, alpha, 0)

    return img

In [ ]:
# %% Enhanced Dataset Generation Pipeline
def generate_dataset():
    """Main dataset generation function with progress tracking and error handling"""
    print("Generating dataset...")

    triangle_counts = []
    triangle_areas = []

    for img_num in tqdm(range(CFG.TOTAL_IMAGES)):
        try:
            img = generate_background()
            label_file = os.path.join(CFG.LABEL_DIR, f"{img_num}.txt")

            if img_num < CFG.TOTAL_IMAGES * CFG.NEGATIVE_RATIO:
                num_triangles = 0
            else:
                weights = [0.5, 0.3, 0.15, 0.05]
                num_triangles = np.random.choice(range(1, CFG.MAX_TRIANGLES), p=weights[:CFG.MAX_TRIANGLES-1])

            triangle_counts.append(num_triangles)

            with open(label_file, 'w') as f:
                for _ in range(num_triangles):
                    triangle = create_triangle()

                    color = get_triangle_color(img)

                    img = draw_triangle_with_effects(img, triangle, color)

                    area = cv2.contourArea(triangle)
                    triangle_areas.append(area)

                    x_min, y_min = np.min(triangle, axis=0)
                    x_max, y_max = np.max(triangle, axis=0)

                    x_center = (x_min + x_max) / (2 * CFG.IMG_SIZE)
                    y_center = (y_min + y_max) / (2 * CFG.IMG_SIZE)
                    width = (x_max - x_min) / CFG.IMG_SIZE
                    height = (y_max - y_min) / CFG.IMG_SIZE

                    f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

            cv2.imwrite(os.path.join(CFG.IMAGE_DIR, f"{img_num}.jpg"), img)

        except Exception as e:
            print(f"Error generating image {img_num}: {e}")
            cv2.imwrite(os.path.join(CFG.IMAGE_DIR, f"{img_num}.jpg"),
                       np.full((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), 128, dtype=np.uint8))
            with open(label_file, 'w') as f:
                pass

    print(f"\nDataset generated with {CFG.TOTAL_IMAGES} images")
    print(f"Negative samples: {int(CFG.TOTAL_IMAGES * CFG.NEGATIVE_RATIO)} ({CFG.NEGATIVE_RATIO*100:.1f}%)")
    print(f"Average triangles per positive image: {np.mean(triangle_counts):.2f}")
    if triangle_areas:
        print(f"Average triangle area: {np.mean(triangle_areas):.1f} px²")
        print(f"Min triangle area: {np.min(triangle_areas):.1f} px²")
        print(f"Max triangle area: {np.max(triangle_areas):.1f} px²")

start_time = time.time()
generate_dataset()
print(f"Dataset generation completed in {time.time() - start_time:.2f} seconds")

!tree {CFG.BASE_DIR} -L 2

In [ ]:
# %% Enhanced Dataset Split and Augmentation
def prepare_dataset_split():
    """Split dataset and create data.yaml file"""
    image_files = sorted([f for f in os.listdir(CFG.IMAGE_DIR) if f.endswith('.jpg')])

    image_paths = [os.path.abspath(os.path.join(CFG.IMAGE_DIR, f)) for f in image_files]

    # Create stratified split based on whether images have triangles or not
    has_triangles = []
    for img_file in image_files:
        label_file = os.path.join(CFG.LABEL_DIR, img_file.replace('.jpg', '.txt'))
        has_triangles.append(os.path.getsize(label_file) > 0)

    # Split dataset ensuring balanced distribution of positive/negative samples
    train_paths, temp_paths = train_test_split(
        list(zip(image_paths, has_triangles)),
        test_size=CFG.TEST_SIZE + CFG.VAL_SIZE,
        stratify=has_triangles,
        random_state=42
    )

    train_paths = [path for path, _ in train_paths]
    temp_images = [path for path, _ in temp_paths]
    temp_has_triangles = [has_triangle for _, has_triangle in temp_paths]

    # Further split temp into val and test
    val_paths, test_paths = train_test_split(
        list(zip(temp_images, temp_has_triangles)),
        test_size=CFG.TEST_SIZE / (CFG.TEST_SIZE + CFG.VAL_SIZE),
        stratify=temp_has_triangles,
        random_state=42
    )

    val_paths = [path for path, _ in val_paths]
    test_paths = [path for path, _ in test_paths]

    # Write split files
    def write_split(file_list, split_name):
        with open(f"{CFG.BASE_DIR}/{split_name}.txt", 'w') as f:
            for file_path in file_list:
                f.write(f"{file_path}\n")
        return len(file_list)

    train_count = write_split(train_paths, 'train')
    val_count = write_split(val_paths, 'val')
    test_count = write_split(test_paths, 'test')

    print(f"Dataset split: {train_count} train, {val_count} validation, {test_count} test images")

    # Create data.yaml
    yaml_content = f"""train: {os.path.abspath(f'{CFG.BASE_DIR}/train.txt')}
val: {os.path.abspath(f'{CFG.BASE_DIR}/val.txt')}
test: {os.path.abspath(f'{CFG.BASE_DIR}/test.txt')}

nc: 1
names: ['triangle']
"""

    with open(f"{CFG.BASE_DIR}/data.yaml", 'w') as f:
        f.write(yaml_content)

    print(f"Created data.yaml at {CFG.BASE_DIR}/data.yaml")

def setup_augmentations():
    """Define a robust augmentation pipeline for YOLOv8"""
    custom_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=CFG.AUG_SCALE_LIMIT,
            rotate_limit=CFG.AUG_ROTATE_LIMIT,
            p=0.5
        ),
        A.RandomBrightnessContrast(p=0.4),
        A.GaussNoise(var_limit=(10, 50), p=0.3),
        A.GaussianBlur(blur_limit=(3, 7), p=0.2),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    return custom_transform

prepare_dataset_split()

In [ ]:
import torch
# %% Enhanced YOLOv8 Training Configuration
def train_model():
    """Configure and train YOLOv8 model with optimized parameters"""
    print("\nInitializing YOLOv8 model...")

    os.makedirs(CFG.SAVE_DIR, exist_ok=True)

    # Initialize the model
    model = YOLO(CFG.MODEL_TYPE)

    training_params = {

        'data': f"{CFG.BASE_DIR}/data.yaml",
        'epochs': CFG.EPOCHS,
        'batch': CFG.BATCH,
        'imgsz': CFG.IMG_SIZE,
        'patience': CFG.PATIENCE,
        'project': CFG.SAVE_DIR,
        'name': 'triangle_detection',
        'exist_ok': True,
        'pretrained': True,
        'optimizer': 'auto',  # AdamW
        'workers': CFG.WORKERS,
        'seed': 42,
        'verbose': True,

        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': CFG.AUG_ROTATE_LIMIT,
        'translate': 0.1,
        'scale': CFG.AUG_SCALE_LIMIT,
        'fliplr': 0.5,
        'flipud': 0.3,
        'mosaic': 0.7,
        'mixup': 0.15,

        'patience': CFG.PATIENCE,

        # Save best model based on validation mAP
        'save_period': -1,

        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'amp': torch.cuda.is_available(),
    }

    print(f"Starting training for {CFG.EPOCHS} epochs...")
    start_time = time.time()

    results = model.train(**training_params)

    training_time = time.time() - start_time
    hours, remainder = divmod(training_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    print(f"Training completed in {int(hours)}h {int(minutes)}m {int(seconds)}s")

    return model, results

trained_model, training_results = train_model()

In [ ]:
def evaluate_model(model):
    """Comprehensive evaluation of the model"""
    print("\nEvaluating model on test set...")

    metrics = model.val(
        data=f"{CFG.BASE_DIR}/data.yaml",
        split='test',
        batch=CFG.BATCH,
        imgsz=CFG.IMG_SIZE,
        conf=0.25,
        iou=0.6,
        plots=True
    )

    print(f"""
{'='*50}
 Final Evaluation Metrics
{'='*50}
mAP@50-95: {metrics.box.map.item():.4f}
mAP@50: {metrics.box.map50.item():.4f}
Precision: {metrics.box.p.item():.4f}
Recall: {metrics.box.r.item():.4f}
F1 Score: {metrics.box.f1.item():.4f}
Inference Speed: {metrics.speed['inference']:.2f}ms/img
Average Processing Time: {metrics.speed['preprocess'] + metrics.speed['inference'] + metrics.speed['postprocess']:.2f}ms/img
""")

    return metrics

In [ ]:
def visualize_results(model, metrics=None):
    """Create comprehensive visualizations of model performance"""
    if CFG.PLOT_METRICS:
        plt.figure(figsize=(20, 12))

        # Plot training curves
        try:
            results_file = f"{CFG.SAVE_DIR}/triangle_detection/results.csv"
            if os.path.exists(results_file):
                import pandas as pd
                results_df = pd.read_csv(results_file)

                plt.subplot(2, 3, 1)
                plt.plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision')
                plt.plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall')
                plt.title('Precision & Recall vs Epoch')
                plt.xlabel('Epoch')
                plt.ylabel('Value')
                plt.legend()
                plt.grid(True, alpha=0.3)

                plt.subplot(2, 3, 2)
                plt.plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP@50')
                plt.plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP@50-95')
                plt.title('mAP vs Epoch')
                plt.xlabel('Epoch')
                plt.ylabel('mAP')
                plt.legend()
                plt.grid(True, alpha=0.3)

                plt.subplot(2, 3, 3)
                plt.plot(results_df['epoch'], results_df['train/box_loss'], label='Train')
                plt.plot(results_df['epoch'], results_df['val/box_loss'], label='Validation')
                plt.title('Box Loss vs Epoch')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.grid(True, alpha=0.3)

                plt.subplot(2, 3, 4)
                plt.plot(results_df['epoch'], results_df['train/cls_loss'], label='Train')
                plt.plot(results_df['epoch'], results_df['val/cls_loss'], label='Validation')
                plt.title('Classification Loss vs Epoch')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.grid(True, alpha=0.3)

                plt.subplot(2, 3, 5)
                plt.plot(results_df['epoch'], results_df['train/dfl_loss'], label='Train')
                plt.plot(results_df['epoch'], results_df['val/dfl_loss'], label='Validation')
                plt.title('DFL Loss vs Epoch')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.grid(True, alpha=0.3)

        except Exception as e:
            print(f"Warning: Could not plot training curves - {e}")

        plt.tight_layout()
        plt.savefig(f"{CFG.SAVE_DIR}/training_curves.png")
        plt.show()

    # Visualize predictions on test samples
    test_files = []
    with open(f"{CFG.BASE_DIR}/test.txt", 'r') as f:
        test_files = f.readlines()

    test_files = [line.strip() for line in test_files]

    if test_files:
        num_samples = min(CFG.TEST_SAMPLES, len(test_files))
        test_images = np.random.choice(test_files, num_samples, replace=False)

        fig, axes = plt.subplots(2, num_samples//2, figsize=(20, 8))
        axes = axes.flatten()

        for i, (ax, img_path) in enumerate(zip(axes, test_images)):
            results = model.predict(img_path, conf=0.25)

            plotted = results[0].plot(line_width=2, font_size=14)

            ax.imshow(cv2.cvtColor(plotted, cv2.COLOR_BGR2RGB))

            num_detections = len(results[0].boxes)
            ax.set_title(f"Sample {i+1}: {num_detections} triangle(s)")
            ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{CFG.SAVE_DIR}/prediction_samples.png")
        plt.show()

    # Visualize triangle distribution
    triangle_counts = []
    for img_num in range(CFG.TOTAL_IMAGES):
        label_file = os.path.join(CFG.LABEL_DIR, f"{img_num}.txt")
        if os.path.exists(label_file):
            with open(label_file, 'r') as f:
                lines = f.readlines()
                triangle_counts.append(len(lines))

    if triangle_counts:
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        sns.histplot(triangle_counts, bins=range(max(triangle_counts)+2), kde=False)
        plt.title('Distribution of Triangles per Image')
        plt.xlabel('Number of Triangles')
        plt.ylabel('Count')
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 2, 2)
        labels = ['No Triangles', '1+ Triangles']
        counts = [triangle_counts.count(0), sum(1 for x in triangle_counts if x > 0)]
        plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title('Positive vs Negative Samples')

        plt.tight_layout()
        plt.savefig(f"{CFG.SAVE_DIR}/dataset_distribution.png")
        plt.show()

metrics = evaluate_model(trained_model)
visualize_results(trained_model, metrics)

In [ ]:
# %% Model Export and Deployment
def export_model():
    """Export the model to ONNX and TorchScript formats"""
    print("\nExporting model...")

    # Path to best model weights
    best_weights = f"{CFG.SAVE_DIR}/triangle_detection/weights/best.pt"

    if os.path.exists(best_weights):
        best_model = YOLO(best_weights)

        formats = ['onnx', 'torchscript']

        for format_type in formats:
            try:
                export_path = best_model.export(format=format_type)
                print(f"Exported model to {export_path}")
            except Exception as e:
                print(f"Failed to export to {format_type}: {e}")

        # Copy best model to Drive for safekeeping
        save_path = f"{CFG.SAVE_DIR}/best_triangle_detector.pt"
        shutil.copy(best_weights, save_path)
        print(f"Saved best model to {save_path}")
    else:
        print(f"Warning: Best model weights not found at {best_weights}")

In [ ]:
# %% Inference Example
def inference_demo():
    """Demonstrate inference on a few test images"""
    print("\nRunning inference demo...")

    # Load best model
    best_model_path = f"{CFG.SAVE_DIR}/triangle_detection/weights/best.pt"
    if not os.path.exists(best_model_path):
        print(f"Model not found at {best_model_path}")
        return

    model = YOLO(best_model_path)

    with open(f"{CFG.BASE_DIR}/test.txt", 'r') as f:
        test_files = [line.strip() for line in f.readlines()]

    if not test_files:
        print("No test files found")
        return

    # Select a few random test images
    sample_size = min(5, len(test_files))
    samples = random.sample(test_files, sample_size)

    print("Running inference on sample images...")

    total_time = 0
    total_triangles = 0

    for img_path in samples:
        start_time = time.time()
        results = model.predict(img_path, conf=0.25, verbose=False)
        inference_time = time.time() - start_time

        total_time += inference_time
        num_triangles = len(results[0].boxes)
        total_triangles += num_triangles

        print(f"Image: {os.path.basename(img_path)} - Detected {num_triangles} triangles in {inference_time*1000:.1f}ms")

    # Calculate average inference time
    avg_time = total_time / sample_size * 1000

    print(f"\nAverage inference time: {avg_time:.2f}ms per image")
    print(f"Average triangles detected: {total_triangles/sample_size:.1f}")
    print(f"FPS: {1000/avg_time:.1f}")

export_model()

inference_demo()

In [ ]:
# %% Keep Colab Session Alive
def keep_alive():
    """Colab session keep-alive function"""
    from IPython.display import Javascript
    display(Javascript('''
    function ClickConnect(){
        console.log("Keeping session alive");
        document.querySelector("colab-connect-button").click()
    }
    setInterval(ClickConnect, 60000)
    '''))

# Uncomment to keep session alive during long runs
# keep_alive()

In [ ]:
# %% Summary and Conclusion
print(f"""
{'='*70}
Triangle Detection Model - Final Summary
{'='*70}

Dataset:
  - Total images: {CFG.TOTAL_IMAGES}
  - Positive/negative ratio: {1-CFG.NEGATIVE_RATIO:.1f}/{CFG.NEGATIVE_RATIO:.1f}
  - Image size: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}
  - Background types: {', '.join(CFG.BG_TYPES)}
  - Triangle types: {', '.join(CFG.TRIANGLE_TYPES)}

Training:
  - Model: YOLOv8 Nano
  - Epochs: {CFG.EPOCHS} with patience {CFG.PATIENCE}
  - Batch size: {CFG.BATCH}
  - Augmentations: Rotation, scaling, flips, color jitter, etc.

Results:
  - Model saved to: {CFG.SAVE_DIR}/triangle_detection/weights/best.pt
  - Exported formats: ONNX, TorchScript

Next Steps:
  1. For deployment, use the exported models
  2. For further improvements:
     - Collect real-world triangle images
     - Fine-tune on domain-specific data
     - Try larger YOLOv8 models (s, m, l) for higher accuracy

Thank you for using the Triangle Detection System!
{'='*70}
""")

In [ ]:
!pip install nbformat --upgrade

In [ ]:
import nbformat
from nbformat import read, write, NO_CONVERT

# Path to your notebook (change if needed)
notebook_path = '/content/your_notebook.ipynb'

# Load the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = read(f, as_version=NO_CONVERT)

# Clear outputs and fix metadata
for cell in nb.cells:
    if 'outputs' in cell:
        cell['outputs'] = []
    if 'execution_count' in cell:
        cell['execution_count'] = None
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    write(nb, f)

print("Notebook cleaned successfully! ✅")